In [1]:
import quandl
import pandas as pd
import numpy as np
from sklearn.svm import SVR # Máquina soporte vectorial 
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.model_selection import GridSearchCV, train_test_split,RandomizedSearchCV #Hiperparametrización
quandl.ApiConfig.api_key = 'DEN8cmx6MYrtzyeUdDeh' #????

In [3]:
%%time
tickers = ['FB','GOOGL','AAPl','AMZN','TSLA','MSFT','AMD','INTC','T','NFLX']
Validate = [] #Conjunto para validación 
Train = [] #Conjunto de entrenamiento

for ticker in tickers:
    df = quandl.get("WIKI/"+ticker)
    #print (df)
    df = df.iloc[-500:]
    func = ['mean','std','skew','kurt'] #Estadísticos de interés
    time = [2,4,7,30,60,120,250] #Días tomados artibatriamente  
    Z = pd.DataFrame()  
    for f in func: 
        for t in time:
            aux = pd.DataFrame()
            for i in range(249):
                aux[ df.index[i+251] ] = list( df.iloc[i-t+251:i+251]['Adj. Close'] ) #
            Z[f+str(t)] = aux.apply(f,axis=0)  
            

    Z.drop(['skew2','kurt2'],axis=1, inplace =True)
    W = pd.merge(Z, df[['Adj. Close']],left_index=True,right_index=True)    
    X = W.drop(['Adj. Close'],1).fillna(0)
    y = W[['Adj. Close']]
    mm1 = MinMaxScaler()
    mm2 = MinMaxScaler()
    mm1.fit(X)
    mm2.fit(y)
    Xm = pd.DataFrame(mm1.transform(X))
    ym = pd.DataFrame(mm2.transform(y))
    Xt, Xv, yt, yv = train_test_split(Xm, ym, test_size = 0.3)
    model =SVR()
    model.fit(Xt,yt)
    # Hiperparametrizando
    param_grid = dict( C = np.arange(0.1,1,0.1),
                  kernel = ['linear', 'poly', 'rbf', 'sigmoid'],
                  degree = range(1,6))
    grid = GridSearchCV(cv=3,
                  verbose=True,
                  estimator=model,
                  n_jobs=-1,
                  param_grid=param_grid)
    grid.fit(Xm,ym)
    model = grid.best_estimator_
    model.fit(Xt,yt)
    print ticker
    print 'ACC Validate {:.2%}'.format(model.score(Xv,yv))
    print 'ACC Train    {:.2%}'.format(model.score(Xt,yt))
    Validate.append(model.score(Xv,yv))
    Train.append(model.score(Xt,yt))
print '\n'
print 'Promedio Validate  {:.2%}'.format(np.array(Validate).mean())
print 'Promedio Train     {:.2%}'.format(np.array(Train).mean())

Fitting 3 folds for each of 180 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:    4.4s finished


FB
ACC Validate 91.92%
ACC Train    94.45%
Fitting 3 folds for each of 180 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Done 400 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:    7.4s finished


GOOGL
ACC Validate 91.37%
ACC Train    92.87%
Fitting 3 folds for each of 180 candidates, totalling 540 fits


...

[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:    6.1s finished


INTC
ACC Validate 96.21%
ACC Train    95.70%
Fitting 3 folds for each of 180 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Done 424 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:    6.0s finished


T
ACC Validate 78.90%
ACC Train    88.15%
Fitting 3 folds for each of 180 candidates, totalling 540 fits
NFLX
ACC Validate 96.27%
ACC Train    96.27%


Promedio Validate  90.22%
Promedio Train     91.68%
CPU times: user 3min 11s, sys: 2.18 s, total: 3min 13s
Wall time: 4min 15s


[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:    5.4s finished


Observación: Los datos que estaban dando como Target son continuos entonces se debía cambiar la SVM en modo regresión 